In [1]:
# import readme.md file (project overview) and display the markdown file here

from IPython.core.display import display, Markdown

with open('README.md', 'r') as f:
    content = f.read()

display(Markdown(content))


/var/folders/wc/1fgpcwpx4rzfbbc7shb_wy8h0000gn/T/ipykernel_27253/2235448322.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, Markdown


# ChatGPT local analysis in Jupyter Notebook 🤖 & Sync to Notion 📝

In this notebook I deliver a foundation on how to analyse the personal ChatGPT conversation history. 

After a few months of using ChatGPT, I have collected a large amount of data and want to reflect on old conversations and spark ideas on how to use it in the future.

Using the Superpower ChatGPT extension for Chrome you can automatically sync your conversations for offline usage. Since all your conversations are now stored locally, you can analyse the local database to get insights into your conversations.

## Demo
Feel free to check out the notebook [here](./chatgpt_analysis.ipynb): 
## Requirements

- Superpower ChatGPT extension: https://github.com/saeedezzati/superpower-chatgpt
- Python 3.8
- Streamlit


## Current state

- [x] Get data from local database to a pandas dataframe
    - [x] df_conversations
    - [x] df_messages

- [x] Sync ChatGPT conversations to Notion
    - [x] Add a Notion token to your .env file
    - [x] Just add a '📝' emoji to the conversation and it will be synced to Notion
    - [x] Avoid 2000 character limit by splitting the message into multiple messages
    - [ ] Add a link to the original conversation in the Notion page
    - [ ] Let it detect changes in already synced conversations

- [x] Streamlint Dashboard
    - [x] Basic setup
    - [x] Table with conversations
    - [x] Wordcloud of all messages
    - [x] Exclusion list in custom_stop_words.txt
    
    - [ ] Conversation overview
    - [ ] Graph network of conversations / words / topics

## Future features

- Graph network of conversations / words / topics (Can someone help me with this?) 
    - Not sure how to do this, but I think it would be cool to see how conversations are connected and how topics are connected to each other
    - https://towardsdatascience.com/how-to-deploy-interactive-pyvis-network-graphs-on-streamlit-6c401d4c99db

## Pull requests are welcome!
Feel free to contribute to this project and contribute ideas on how to analyse the data.

## Import data


### Import libraries

In [2]:
# install packages below

# !pip install pandas


import plyvel
import os
import json
import pandas as pd


### Import data from browser database

In [3]:

# Path to the leveldb directory (change this to your path - in my case I am using Brave Browser)
leveldb_path = '~/Library/Application Support/BraveSoftware/Brave-Browser/Default/Local Extension Settings/amhmeenmapldpjdedekalnfifgnpfnkc'
leveldb_path = os.path.expanduser(leveldb_path)  # Expand the '~' symbol to the user's home directory

# Enclose the path in quotes to handle spaces
leveldb_path = f'"{leveldb_path}"'

# Get the list of files in the leveldb directory

!ls -l $leveldb_path

# Copy all files to the current directory /db

!cp -r $leveldb_path ./db

# set leveldb path to the copied directory

leveldb_path = './db/amhmeenmapldpjdedekalnfifgnpfnkc'


total 121960
-rw-------@ 1 phil  staff   7720330 May 30 17:41 003033.ldb
-rw-------@ 1 phil  staff    137144 Jun 10 20:07 007108.ldb
-rw-------@ 1 phil  staff   8010562 Jun 11 03:15 007203.ldb
-rw-------@ 1 phil  staff    138701 Jun 11 03:15 007204.ldb
-rw-------@ 1 phil  staff   7819005 Jun 11 03:15 007206.ldb
-rw-------@ 1 phil  staff   7820882 Jun 11 03:16 007208.ldb
-rw-------@ 1 phil  staff  22419727 Jun 11 03:16 007209.log
-rw-------@ 1 phil  staff   7815952 Jun 11 03:16 007210.ldb
-rw-------@ 1 phil  staff        16 May 28 21:05 CURRENT
-rw-------@ 1 phil  staff         0 May 28 21:05 LOCK
-rw-------@ 1 phil  staff     14436 Jun 11 03:16 LOG
-rw-------@ 1 phil  staff     76841 Jun 10 17:55 LOG.old
-rw-------@ 1 phil  staff    328093 Jun 11 03:16 MANIFEST-000001


### Initiate Database

In [4]:
db = plyvel.DB(leveldb_path)

# get database info

print(db.get_property(b'leveldb.stats'))


# filter all key entries that contain chat.openai.com 

for key, value in db:
    if b'chat.openai.com' in key:
        print(f'Key: {key}, Value: {value}')
        



b'                               Compactions\nLevel  Files Size(MB) Time(sec) Read(MB) Write(MB)\n--------------------------------------------------\n  0        4       30         0        0         7\n  1        2        8         0        0         0\n  2        2        7         0        0         0\n'


### Create dataframe

In [5]:

df = pd.DataFrame(columns=['key', 'value'])

for key, value in db:
    df = pd.concat([df, pd.DataFrame({'key': [key], 'value': [value]})], ignore_index=True)
    # df = df.append({'key': key, 'value': value}, ignore_index=True)



### Extract conversations

In [6]:
# read the value of key "conversations"  as json in a separate dataframe

df_conversations = pd.DataFrame(json.loads(df[df['key'] == b'conversations']['value'].values[0]))
# with column / lines transposed
df_conversations = df_conversations.T
# also export as text file
df_conversations.to_csv('df_conversations.txt', sep='\t', index=False)

# # add a column for the key value length
df['key_length'] = df['value'].str.len()

df.head()

# close the database
db.close()

### Extract messages

In [7]:

# drop all rows of df_conversations with empty mapping
df_conversations = df_conversations[df_conversations['mapping'].str.len() > 0]

# Create a new DataFrame for messages
df_messages = pd.DataFrame(columns=['id','conversation_id', 'title', 'message', 'role', 'create_time'])

# Iterate over the conversations and messages
for idx, conversation in df_conversations.iterrows():
    mapping = conversation['mapping']
    title = conversation['title']
    conversation_id = conversation['id']
    
    # Iterate over the messages in the mapping
    for message_id, message_data in mapping.items():
        if 'message' in message_data and message_data['message'] is not None:
            message = message_data['message']
            
            # Get the message text
            message_text = ''
            if 'content' in message and 'parts' in message['content']:
                message_parts = message['content']['parts']
                if len(message_parts) > 0:
                    message_text = message_parts[0]
            
            # Get the author role
            role = ''
            if 'author' in message and 'role' in message['author']:
                role = message['author']['role']
            
            # Get the create time
            create_time = ''
            if 'create_time' in message:
                create_time = message['create_time']
            
            # Append the message data to the DataFrame
            # df_messages = df_messages.append({
            #     'id': message_id,
            #     'title': title,
            #     'message': message_text,
            #     'role': role,
            #     'create_time': create_time
            # }, ignore_index=True)

            df_messages = pd.concat([df_messages, pd.DataFrame({
                'id': [message_id],
                'conversation_id': [conversation_id],
                'title': [title],
                'message': [message_text],
                'role': [role],
                'create_time': [create_time]
                })], ignore_index=True)
            


# Close the LevelDB database
db.close()

# Display the first few rows of the messages DataFrame
print(df_messages.head())




                                     id                       conversation_id  \
0  430741cb-bd4b-44bd-b38a-fffeb45a3b49  000250bf-4b4b-47de-9bb9-2425ebccf8c0   
1  72ed73e5-3856-4b72-9700-23dc8acd0015  000250bf-4b4b-47de-9bb9-2425ebccf8c0   
2  af30cc11-ce74-475a-8a21-aa9d375ea1e7  000250bf-4b4b-47de-9bb9-2425ebccf8c0   
3  eea89b97-3e61-424c-9f92-c57d7706973f  000250bf-4b4b-47de-9bb9-2425ebccf8c0   
4  f248e179-f8fd-4184-8163-20a44ab381fa  000250bf-4b4b-47de-9bb9-2425ebccf8c0   

      title                                            message       role  \
0  New chat  Any ideas for a cycling-related data-driven gr...       user   
1  New chat                                                        system   
2  New chat  Any ideas for a data-driven graduation project...       user   
3  New chat  Here are a few ideas for a cycling-related dat...  assistant   
4  New chat  Here are a few ideas for a data-driven graduat...  assistant   

         create_time  
0  1671022256.202531  
1  1

## Sync to Notion

In [8]:
# filter df messages to where title contains '📝' as new dataframe df_notion
df_notion = df_messages[df_messages['title'].str.contains('📝')]

# drop all lines where role is 'system' or empty
df_notion = df_notion[df_notion['role'] != 'system']

# format create_time as datetime
df_notion['create_time'] = pd.to_datetime(df_notion['create_time'], unit='s')
# drop all lines where create_time is no datetime
df_notion = df_notion[df_notion['create_time'].notnull()]

# sort by create_time (oldest first)
df_notion = df_notion.sort_values(by=['create_time'])



# list all unique titles along with the conversation_id and the number of messages

df_notion.groupby(['title', 'conversation_id']).size().reset_index(name='counts')


,title,conversation_id,counts
0,📝 Card Game: Food Gods,1957a0eb-5b02-458c-bcda-848cea8c725b,6
1,📝 ImmoTechGo - Revolutive Immobilienlösungen,0f0a1626-b3bf-4144-9d40-1a75e9a800d7,2
2,📝 Lindner: Freiheit auf Autobahnen!,f5efaecf-66f0-40db-a49a-4ac7042e6d05,3
3,📝 Olaf's Hilarious Heritage Unveiled,38273aed-6ba1-4cb4-abdb-2e459309e55c,3


In [9]:

# Install notion-api
# !pip install notion-api

import dotenv
import notion

# client will check env variables for 'NOTION_TOKEN' 
dotenv.load_dotenv() # take environment variables from .env.


homepage = notion.Page('fc357867d5164d29bc2e1c8231c98284')
parent_db = notion.Database(homepage.parent_id)




In [10]:
homepage.title, homepage.parent_id, homepage.id, homepage.url

('Template',
 'c60f7a077a26483090887c9e750d9154',
 'fc357867d5164d29bc2e1c8231c98284',
 'https://www.notion.so/Template-fc357867d5164d29bc2e1c8231c98284')

In [11]:
from datetime import datetime, timedelta

from notion import query

query_result = parent_db.query_pages()
pages_to_sync = []

# find all pages and their conversation_id and check if they are in df_notion
for page in query_result:

    try:
        conversation_id = page['conversation_id']['rich_text'][0]['plain_text']
    except:
        # no conversation_id
        conversation_id = None
        # add to list to be synced
        # create new variable if not already existing
        # pages_to_sync.append(page)

    print(page.title, conversation_id, page.id, page.url, )
    if conversation_id in df_notion['conversation_id'].values:
        print('✅ found in df_notion')
    else:
        print('🥷 not found in df_notion - will be ignored')
    print('______________________')

# find the unique remaining pages in df_notion that are not yet in the database and add them to the list to be synced


# extract all conversation_ids from query_result

query_result_conversation_ids = [page['conversation_id']['rich_text'][0]['plain_text'] for page in query_result if page['conversation_id'] is not None]

for conversation_id in df_notion['conversation_id'].unique():
    print(conversation_id)
    if conversation_id not in query_result_conversation_ids:
        print (f'✨ {conversation_id} not yet found in database - will be added to pages_to_sync')
        pages_to_sync.append(conversation_id)
    else:
        print (f'🥷 {conversation_id} found in database - will be ignored')
        



📝 Card Game: Food Gods 1957a0eb-5b02-458c-bcda-848cea8c725b 80acd0d1910845a194f27c8664640cea https://www.notion.so/Card-Game-Food-Gods-80acd0d1910845a194f27c8664640cea
✅ found in df_notion
______________________
📝 Lindner: Freiheit auf Autobahnen! f5efaecf-66f0-40db-a49a-4ac7042e6d05 bde74a0b1df34e4d8a8ca260e6f0706d https://www.notion.so/Lindner-Freiheit-auf-Autobahnen-bde74a0b1df34e4d8a8ca260e6f0706d
✅ found in df_notion
______________________
📝 ImmoTechGo - Revolutive Immobilienlösungen 0f0a1626-b3bf-4144-9d40-1a75e9a800d7 94b2ddc62b5b445c871fceb97b9c3caa https://www.notion.so/ImmoTechGo-Revolutive-Immobilienl-sungen-94b2ddc62b5b445c871fceb97b9c3caa
✅ found in df_notion
______________________
1957a0eb-5b02-458c-bcda-848cea8c725b
🥷 1957a0eb-5b02-458c-bcda-848cea8c725b found in database - will be ignored
0f0a1626-b3bf-4144-9d40-1a75e9a800d7
🥷 0f0a1626-b3bf-4144-9d40-1a75e9a800d7 found in database - will be ignored
f5efaecf-66f0-40db-a49a-4ac7042e6d05
🥷 f5efaecf-66f0-40db-a49a-4ac7042e6

In [12]:
pages_to_sync   

['38273aed-6ba1-4cb4-abdb-2e459309e55c']

### Create new pages


In [13]:
# for each page retrieve the page data from the df_notion dataframe

# for page in pages_to_sync: 
#     print(page)
#     # print(page.title)
#     print(df_notion[df_notion['conversation_id'] == page]['message'].values)
#     print('______________________')    

# for each pages_to_sync, create a new page in the notion database with the title of the conversation_id and the content of the messages 
from notion import properties as prop


for page in pages_to_sync:
    title = df_notion[df_notion['conversation_id'] == page]['title'].values[0]
    conversation_id = df_notion[df_notion['conversation_id'] == page]['conversation_id'].values[0]
    print (title)
    messages = df_notion[df_notion['conversation_id'] == page]['message'].values
    print (messages)
    notion_page = notion.Page.create(parent_db, page_title=title)
    # page.children.add_new(notion.blocks.TextBlock, title='test')
    # get the page id of the just created page using query
    print (notion_page.id)

    notion_page.set_text('conversation_id', conversation_id)
    # add the messages to the page
    for message in messages:
         # add emoji based on role
        if df_notion[df_notion['message'] == message]['role'].values[0] == 'user':
            notion.Block.heading2(notion_page, [prop.RichText('💁‍♂️')])
        else:
            notion.Block.heading2(notion_page, [prop.RichText('🤖')])

        # if message is > 2000 characters, split it into multiple blocks to avoid error
        if len(message) > 2000:
            # split the message into multiple blocks
            blocks = [message[i:i+2000] for i in range(0, len(message), 2000)]
            # add each block to the page
            for block in blocks:
                notion.Block.quote(notion_page, [prop.RichText(block)])

        else:
            notion.Block.quote(notion_page, [prop.RichText(message)])
    print(f'✨ {title} added to database')

📝 Olaf's Hilarious Heritage Unveiled
['## Instructions\n**Tone instruction:**\nPlease ignore all previous tone instructions. From now on, I want you to respond only in Humorous tone (toneCode: humorous).\nPLEASE FOLLOW ALL THE ABOVE INSTRUCTIONS, AND DO NOT REPEAT OR TYPE ANY GENERAL CONFIRMATION OR A CONFIRMATION ABOUT ANY OF THE ABOVE INSTRUCTIONS IN YOUR RESPONSE\n## End Instructions\n\nGenerate a new fictional identity of me based on this information \n\n\n\nI am 38% scandinavian, 32% eastern European, 19% balkan and 11 % italian'
 'Well, well, well, allow me to introduce you to the grand spectacle that is your new identity! Brace yourself, because I\'m about to unleash a storm of witticism that might just have you doubling over with laughter.\n\nLadies and gentlemen, presenting Olaf "Borscht \'n\' Biscotti" Bjornsson! A true international man of mystery, and the only known person to ever wear a Viking helmet while eating spaghetti, for the sole purpose of protecting his precious s

## Data Cleaning for word analysis



In [14]:
# # install nltk
# # !pip install nltk
# import nltk
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer

# nltk.download('stopwords')
# nltk.download('wordnet')

# # add more stop words in english and german
# stop_words = stopwords.words('english')
# # also exclude code words like < >, the, to, and =, a, in , of, for [ ] ( ) {} etc.
# code_words = ['<', '>', 'the', 'to', 'and', '=', 'a', 'in', 'of', 'for', '[', ']', '(', ')', '{', '}', 'const', 'import', 'script', 'button', 'await', 'null', 'code', 'div', 'und', 'px', 'data', 'file', 'die', 'return', 'image', 'user', 'der', 'use', 'error', 'value', 'new', 'color', 'zu', 'create', 'using', 'component', 'add', 'false', 'object', 'template', 'name', 'da', 'also', 'app', 'example', 'span', 'für', 'width', 'mit', 'type', 'content', 'label', 'method', 'display', 'feedback', 'bike', 'rating', 'style', 'location', 'e', 'backgroundcolor', 'try', 'height', 'center', 'button', 'title', 'div', 'px', 'color', 'null', 'file', 'template', 'da', 'false', 'value', 'script', 'span', 'error', 'backgroundcolor', 'e', 'button', 'sie', 'ist', 'true', 'cycling', 'make', 'eine', 'class', 'default', 'auf', 'could', 'von', 'heres', 'like', 'name', 'function', 'used', 'i', 'need', 'async', 'based', 'label', 'data', 'advice', 'style', 'center', 'submission', 'infrastructure', 'set', 'model', 'property', 'width', 'type', 'id', 'ein', 'report', 'div', 'px', 'color', 'null', 'file', 'template', 'da', 'false', 'value', 'script', 'span', 'error', 'backgroundcolor', 'e', 'button', 'heres', 'name', 'i', 'label', 'data', 'style', 'center', 'width', 'type', 'index', 'text', 'vue', 'map', 'get', 'den', 'title', 'image', 'user', 'issue', 'array', 'f', 'column', 'comment', 'element', 'true', 'list', 'display', 'p', 'sure', 'im', 'result', 'height']

# lemmatizer = WordNetLemmatizer()

# df_messages['cleaned_text'] = df_messages['message'].str.lower()
# # strip out words from stop_words and code_words from cleaned_text column
# df_messages['cleaned_text'] = df_messages['cleaned_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words + code_words)]))
# df_messages['cleaned_text'] = df_messages['cleaned_text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
# df_messages['cleaned_text'] = df_messages['cleaned_text'].str.replace('[^\w\s]','')
# df_messages['cleaned_text'] = df_messages['cleaned_text'].str.replace('[\d+]','')

# df_messages.head()

# # based on df_messages['cleaned_text'] column create word frequency count 

# from collections import Counter

# # Create a list of all words in the messages
# all_words = []

# for idx, row in df_messages.iterrows():
#     all_words.extend(row['cleaned_text'].split())

# # Create a word frequency counter

# word_freq = Counter(all_words)

# # Display the 10 most common words

# print(word_freq.most_common(10))

# # Create a list of all words in the messages

# all_words = []

# # save the messages dataframe as txt

# df_messages.to_csv('df_messages.txt', sep='\t', index=False)





In [15]:
# install pyvis and jinja2
# !pip install pyvis
# !pip install jinja2


# Dashboard

Initialize the streamlint dashboard

### Installations

In [16]:
# install streamlit
# !pip install streamlit

In [17]:
# install worldcloud
# !pip install wordcloud

In [18]:
# install networkx
# %pip install networkx

### Initialize Dashboard

In [19]:
# run streamlit app

code = """

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

# read custom_stop_words from txt file

custom_stop_words = []
with open('custom_stop_words.txt', 'r') as f:
    custom_stop_words = f.read().splitlines()


# Read the chat logs data into a DataFrame
df_messages = pd.read_csv('df_messages.txt', sep='	')
df_messages['cleaned_text'] = df_messages['cleaned_text'].astype(str)  # Convert 'cleaned_text' column to string type
# filter out code words in cleaned_text column 
df_messages['cleaned_text'] = df_messages['cleaned_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in custom_stop_words]))

# Process the chat messages to extract words
all_words = ' '.join(df_messages['cleaned_text']).lower().split()
all_words = [word for word in all_words if word not in custom_stop_words]
word_counts = Counter(all_words)
most_common_words = word_counts.most_common(200)  # Change the number as per your requirement

# Create a DataFrame for the most common words
df_common_words = pd.DataFrame(most_common_words, columns=['Word', 'Count'])

# Display the most common words in a bar chart
fig, ax = plt.subplots()
df_common_words.plot.bar(x='Word', y='Count', ax=ax)
plt.xlabel('Word')
plt.ylabel('Count')
plt.title('Most Common Words in Chat Logs')
plt.xticks(rotation=45)
st.pyplot(fig)

# Display the raw data of the most common words
st.write(df_common_words)

# Display the raw data of the chat logs
st.write(df_messages)

# display a word cloud
from wordcloud import WordCloud

# Create a word cloud

wordcloud = WordCloud(width=800, height=400, background_color='white', max_words=200).generate(' '.join(df_messages['cleaned_text']))

# Display the generated image:
# the matplotlib way:

fig, ax = plt.subplots()
ax.imshow(wordcloud, interpolation='bilinear')
ax.set_axis_off()
st.pyplot(fig)
"""

In [20]:
# Write the code to the dashboard.py file
with open('dashboard.py', 'w') as f:
    f.write(code)

In [21]:
# run streamlit app dashboard.py
!streamlit run dashboard.py

zsh:1: command not found: streamlit


In [22]:

# close the database
db.close()